In [2]:
import os
import json
from pycocotools.coco import COCO

## 这里改成数据集路径
dataset_dir = "/dataroot/coco"

coco_instance_json_file = os.path.join(dataset_dir, "annotations/instances_val2014.json")

instance_info = {}
with open(coco_instance_json_file, 'r') as f:
    instance_info = json.load(f)

clsid2clsidx = {}
clsidx2clsid = {}
clsid2clsname = {}
for idx, cat_info in enumerate(instance_info["categories"]):
    clsid2clsidx[cat_info['id']] = idx
    clsidx2clsid[idx] = cat_info['id']
    clsid2clsname[cat_info['id']] = cat_info['name']

coco_val = COCO(coco_instance_json_file)
# valset_ids = coco_val.getImgIds()

coco_train = COCO(os.path.join(dataset_dir, "annotations/instances_train2014.json"))
# valset_ids = coco_train.getImgIds()


loading annotations into memory...
Done (t=2.98s)
creating index...
index created!
loading annotations into memory...
Done (t=8.79s)
creating index...
index created!


In [16]:
def get_coco_train(id):
    path_ = os.path.join(dataset_dir, 'train2014/{}'.format(coco_train.loadImgs(ids = id)[0]['file_name']))
    
    label_list = [0] * 80
    annIds = coco_train.getAnnIds(imgIds = id)
    anns = coco_train.loadAnns(annIds)
    for ann in anns:
        tmp_idx = clsid2clsidx[ann['category_id']]
        label_list[tmp_idx] = 1
    return path_, label_list

def get_coco_val(id):
    path_ = os.path.join(dataset_dir, 'val2014/{}'.format(coco_val.loadImgs(ids = id)[0]['file_name']))
    
    label_list = [0] * 80
    annIds = coco_val.getAnnIds(imgIds = id)
    anns = coco_val.loadAnns(annIds)
    for ann in anns:
        tmp_idx = clsid2clsidx[ann['category_id']]
        label_list[tmp_idx] = 1
    return path_, label_list

In [29]:
voc_object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
                        'bottle', 'bus', 'car', 'cat', 'chair',
                        'cow', 'diningtable', 'dog', 'horse',
                        'motorbike', 'person', 'pottedplant',
                        'sheep', 'sofa', 'train', 'tvmonitor']

def read_object_labels(path):
    path_labels = os.path.join(path, 'ImageSets', 'Main')
    labeled_data = dict()
    num_classes = len(voc_object_categories)

    for j, phase in enumerate(['trainval', 'test']):
        for i in range(num_classes):
            file = os.path.join(path_labels, voc_object_categories[i] + '_' + phase + '.txt')
            data_ = read_image_label(file)

            for (name, label) in data_.items():
                if name in labeled_data:
                    labeled_data[name][i] = 1 if label==1 else 0
                else:
                    labels = [0] * num_classes
                    labels[i] = 1 if label==1 else 0
                    labeled_data[name] = labels

    return labeled_data

def read_image_label(file):
    print('[dataset] read ' + file)
    data_ = dict()
    with open(file, 'r') as f:
        for line in f:
            tmp = line.strip().split(' ')
            name = tmp[0]
            label = int(tmp[-1])
            data_[name] = label
    return data_

def voc2coco_label(voc_label):
    tmp = [5, 2, 15, 9, 40, 6, 3, 16, 57, 20, 61, 17, 18, 4, 1, 59, 19, 58, 7, 63]   
    voc2coco_idx = {i: j-1 for i, j in enumerate(tmp)}
    
    ret = [0] * 80
    for i in range(len(voc_label)):
        if voc_label[i] == 1:
            ret[voc2coco_idx[i]] = 1
    return ret

## 替换路径
dataset_dir_voc = "/dataroot/VOCdevkit/VOC2007"
imid_to_labels = read_object_labels(dataset_dir_voc)
dataset_dir_voc_img = os.path.join(dataset_dir_voc, "JPEGImages")

def get_voc(id):
    cur_name = str(id).zfill(6)
    path_ = os.path.join(dataset_dir_voc_img, '{}.jpg'.format(cur_name))
    label_list = imid_to_labels[cur_name]
    return path_, voc2coco_label(label_list)

[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/aeroplane_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/bicycle_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/bird_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/boat_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/bottle_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/bus_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/car_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/cat_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/chair_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/cow_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/diningtable_trainval.txt
[dataset] read /mnt/HDD/data/VOCdevkit/VOC2007/ImageSets/Main/dog_trainval.txt
[dataset] read /mnt/HDD/dat

In [30]:
## 替换成需要读取的文件
with open('data_source_cs_hit.txt', 'r') as f: 
    data_id_list = f.readlines()

# with open('data_source_testvoc.txt', 'r') as f:
#     data_id_list = f.readlines()

imnames_list = []
imlabels_list = []
for data_info in data_id_list:
    data_info = data_info.strip()
    if data_info:
        data_source, data_id = data_info.split(' ')
        data_id = int(data_id)
        if data_source.lower() == 'coco_train':
            cur_root, cur_label = get_coco_train(data_id)
        elif data_source.lower() == 'coco_val':
            cur_root, cur_label = get_coco_val(data_id)
        elif data_source.lower() == 'voc':
            cur_root, cur_label = get_voc(data_id)
        else:
            raise KeyError('data_source.lower()')
        imnames_list.append(cur_root)
        imlabels_list.append(cur_label)

#### 全部整理完成之后写入json文件

In [ ]:
import json

with open('imnames.json', 'w') as f:
    json.dump(imnames_list ,f)

with open('imlabels.json', 'w') as f:
    json.dump(imlabels_list ,f)